In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import export_text

In [38]:
con = sqlite3.connect("Dataset/database.db")

awards_players_df = pd.read_sql_query("SELECT * FROM awards_players", con)
coaches_df = pd.read_sql_query("SELECT * FROM coaches", con)
players_teams_df = pd.read_sql_query("SELECT * FROM players_teams", con)
players_df = pd.read_sql_query("SELECT * FROM players", con)
series_post_df = pd.read_sql_query("SELECT * FROM series_post", con)
#teams_post_df = pd.read_sql_query("SELECT * FROM teams_post", con)
teams_df = pd.read_sql_query("SELECT * FROM teams", con)


teams_df = teams_df.tail(-1)
teams_df = teams_df.drop(['lgID', 'rank','seeded','firstRound','semis', 'finals','GP','arena','name','tmORB','tmDRB','tmTRB','opptmORB','opptmDRB','opptmTRB','divID'], axis=1)
teams_df = pd.get_dummies(teams_df, columns=['tmID', 'franchID', 'confID'])


teams_df.head()





,year,playoff,o_fgm,o_fga,o_ftm,o_fta,o_3pm,o_3pa,o_oreb,o_dreb,...,franchID_MIN,franchID_NYL,franchID_PHO,franchID_POR,franchID_SAC,franchID_SAS,franchID_SEA,franchID_WAS,confID_EA,confID_WE
1,9,N,895,2258,542,725,202,598,340,737,...,False,False,False,False,False,False,False,False,True,False
2,10,Y,1089,2428,569,755,114,374,404,855,...,False,False,False,False,False,False,False,False,True,False
3,1,N,812,1903,431,577,131,386,305,630,...,False,False,False,False,False,False,False,False,True,False
4,2,Y,746,1780,410,528,153,428,309,639,...,False,False,False,False,False,False,False,False,True,False
5,3,Y,770,1790,490,663,211,527,302,653,...,False,False,False,False,False,False,False,False,True,False


In [39]:
teams_df["playoff"] = teams_df["playoff"].map({"N": 0, "Y": 1})

X = teams_df.drop('playoff', axis=1)
y = teams_df['playoff']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy: .2f}")

print(classification_report(y_test, y_pred))

tree_rules = export_text(clf, feature_names=list(X.columns))
print("Decision Tree Rules:")
print(tree_rules)


Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.79      0.73      0.76        15
           1       0.86      0.89      0.88        28

    accuracy                           0.84        43
   macro avg       0.82      0.81      0.82        43
weighted avg       0.84      0.84      0.84        43

Decision Tree Rules:
|--- won <= 16.50
|   |--- confL <= 9.50
|   |   |--- class: 1
|   |--- confL >  9.50
|   |   |--- o_3pm <= 242.50
|   |   |   |--- d_pf <= 770.00
|   |   |   |   |--- class: 0
|   |   |   |--- d_pf >  770.00
|   |   |   |   |--- class: 1
|   |   |--- o_3pm >  242.50
|   |   |   |--- class: 1
|--- won >  16.50
|   |--- d_3pa <= 358.00
|   |   |--- class: 0
|   |--- d_3pa >  358.00
|   |   |--- awayL <= 12.50
|   |   |   |--- tmID_UTA <= 0.50
|   |   |   |   |--- class: 1
|   |   |   |--- tmID_UTA >  0.50
|   |   |   |   |--- attend <= 106974.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- attend >  106974.50
|   |   | 